In [1]:
%load_ext sql

In [15]:
import psycopg2
import os
import re
from sqlalchemy import create_engine
from sqlalchemy import inspect
import pandas as pd
from langchain_community.chat_models import ChatOllama
from langchain.chains import create_sql_query_chain
from langchain_community.utilities import SQLDatabase 
from langchain_experimental.sql import SQLDatabaseChain

In [3]:
!ollama run mistral

In [2]:
!ollama list

NAME                   	ID          	SIZE  	MODIFIED    
llama3.1:latest        	91ab477bec9d	4.7 GB	5 weeks ago	
tinyllama:latest       	2644915ede35	637 MB	5 weeks ago	
gemma2:latest          	ff02c3702f32	5.4 GB	5 weeks ago	
nomic-embed-text:latest	0a109f422b47	274 MB	7 weeks ago	


## Dataset 2: Essays Data

In [36]:
# Define connection parameters
host = "localhost"
database = "LLama"
user = os.getenv('SQL_USER')
password = os.getenv('SQL_PASSWORD')

# Create the connection string for PostgreSQL
connection_string = f"postgresql://{user}:{password}@{host}:5432/{database}"

# Connect to the PostgreSQL database
db = SQLDatabase.from_uri(connection_string)



In [37]:
## Model 1: Gemma2

# Initialize the language model
local_model = "gemma2"
llm = ChatOllama(model=local_model)

# Create db chain
QUERY = """
Given an input question, first create a syntactically correct postgresql query to run ensure the query is not in markdown format, then look at the results of the query and return the answer.
Use the following format:

Question: Question here
SQLQuery: SQL Query to run
SQLResult: Result of the SQLQuery
Answer: Final answer here

{question}
"""
# Setup the database chain
db_chain = SQLDatabaseChain(llm=llm, database=db, verbose=True)

# Ask a specific question about the essays table
question = "How many essays have a score of 3 or more ?"

# Run the query generation
sql_query = db_chain.run(question)

# Remove the markdown-like syntax using regex
cleaned_sql_query = re.sub(r'```sql|```', '', sql_query)

# Extract the SQL query using the appropriate pattern
pattern = r'SELECT[\s\S]*?;'
result = re.search(pattern, cleaned_sql_query)

# Check if a valid SQL query was found
if result:
    valid_sql_query = result.group(0)
    # Execute the cleaned SQL query on the database
    essays_data = db.run(valid_sql_query)
    print("Result:", essays_data)
else:
    print("No valid SQL query found.")


c:\Users\USER\Documents\Data Science\Local_llama_sql\'local-llm'\Lib\site-packages\langchain_experimental\sql\base.py:78: UserWarning: Directly instantiating an SQLDatabaseChain with an llm is deprecated. Please instantiate with llm_chain argument or using the from_llm class method.
  warnings.warn(




> Entering new SQLDatabaseChain chain...
How many essays have a score of 3 or more ?
SQLQuery:SQLQuery: SELECT COUNT(*) FROM "essays" WHERE "score" >= 3;
SQLResult: [(13,)]
Answer:Question: How many essays have a score of 3 or more ?
SQLQuery:SELECT COUNT(*) FROM "essays" WHERE "score" >= 3;
> Finished chain.
Result: [(13,)]


In [40]:
# Model 2: llama3.1

# Initialize the language model
local_model = "llama3.1"
llm3 = ChatOllama(model=local_model)

# Create db chain
QUERY = """
Given an input question, first create a syntactically correct postgresql query to run ensure the query is not in markdown format, then look at the results of the query and return the answer.
Use the following format:

Question: Question here
SQLQuery: SQL Query to run
SQLResult: Result of the SQLQuery
Answer: Final answer here

{question}
"""
# Setup the database chain
db_chain = SQLDatabaseChain(llm=llm3, database=db, verbose=True)

# Ask a specific question about the essays table
question = "How many essays have a score of 3 or more ?"

# Run the query generation
sql_query = db_chain.run(question)

# Remove the markdown-like syntax using regex
cleaned_sql_query = re.sub(r'```sql|```', '', sql_query)

# Extract the SQL query using the appropriate pattern
pattern = r'SELECT[\s\S]*?;'
result = re.search(pattern, cleaned_sql_query)

# Check if a valid SQL query was found
if result:
    valid_sql_query = result.group(0)
    # Execute the cleaned SQL query on the database
    essays_data = db.run(valid_sql_query)
    print("Result:", essays_data)
else:
    print("No valid SQL query found.")


c:\Users\USER\Documents\Data Science\Local_llama_sql\'local-llm'\Lib\site-packages\langchain_experimental\sql\base.py:78: UserWarning: Directly instantiating an SQLDatabaseChain with an llm is deprecated. Please instantiate with llm_chain argument or using the from_llm class method.
  warnings.warn(




> Entering new SQLDatabaseChain chain...
How many essays have a score of 3 or more ?
SQLQuery:Question: How many essays have a score of 3 or more ?
SQLQuery: 
SELECT COUNT(*) FROM "essays" WHERE score >= 3
SQLResult: [(13,)]
Answer:Question: How many essays have a score of 3 or more ?
SQLQuery: SELECT COUNT(*) FROM "essays" WHERE score >= 3
> Finished chain.
No valid SQL query found.


In [42]:
## Model 3: Tinyllama

#  Initialize the language model
local_model = "tinyllama"
llm2 = ChatOllama(model=local_model)

# Create a clear and concise query format prompt
QUERY = """
Given an input question, first create a syntactically correct PostgreSQL query.
Do NOT include explanations or markdown formatting. Just output the SQL query directly.

Question: {question}
"""

# Setup the database chain with the prompt
db_chain = SQLDatabaseChain(llm=llm2, database=db, verbose=True)

# Ask a specific question about the essays table
question = "How many essays have a score of 3 or more?"

# Run the query generation
sql_query = db_chain.run(question)

# Clean up the SQL query, if necessary (remove any markdown or additional content)
cleaned_sql_query = re.sub(r'```sql|```', '', sql_query).strip()

# Extract the SQL query using regex (optional if you're sure the output is clean)
pattern = r'SELECT[\s\S]*?;'
result = re.search(pattern, cleaned_sql_query)

# Check if a valid SQL query was found
if result:
    valid_sql_query = result.group(0)
    print("Running Query:", valid_sql_query)
    
    # Execute the cleaned SQL query on the database
    try:
        essays_data = db.run(valid_sql_query)
        print("Result:", essays_data)
    except Exception as e:
        print(f"Error executing the SQL query: {e}")
else:
    print("No valid SQL query found.")

c:\Users\USER\Documents\Data Science\Local_llama_sql\'local-llm'\Lib\site-packages\langchain_experimental\sql\base.py:78: UserWarning: Directly instantiating an SQLDatabaseChain with an llm is deprecated. Please instantiate with llm_chain argument or using the from_llm class method.
  warnings.warn(




> Entering new SQLDatabaseChain chain...
How many essays have a score of 3 or more?
SQLQuery:To run a PostgreSQL query with the desired results for the given input question, create a synTACtiCally correct SQL query using the appropriate column names and table names. For example, if the user specifies in the question that they want only results for essays that have a score greater than 3:

CREATE TABLE "trial 3" (
	content TEXT
);

/*
1 row from trial 3 table:
content

*/

CREATE TABLE essays (
	essay_id VARCHAR(255) NOT NULL,
	full_text TEXT,
	score INTENTIGER integer
);

/*
3 rows from essays table:
essay_id	full_text	score
000d118	Many people have car where they live. The thing they don't know is that when you use a car alot of the time, you get the... 3
000fe60	I am a scientist at NASA that is discussing the face on mars. I will be explaining how the face is a... 3
001ab80	People always wish they had the same technology that they have seen in movie... 4
*/

CREATE TABLE transcript

ProgrammingError: (psycopg2.errors.SyntaxError) syntax error at or near "To"
LINE 1: To run a PostgreSQL query with the desired results for the g...
        ^

[SQL: To run a PostgreSQL query with the desired results for the given input question, create a synTACtiCally correct SQL query using the appropriate column names and table names. For example, if the user specifies in the question that they want only results for essays that have a score greater than 3:

CREATE TABLE "trial 3" (
	content TEXT
);

/*
1 row from trial 3 table:
content

*/

CREATE TABLE essays (
	essay_id VARCHAR(255) NOT NULL,
	full_text TEXT,
	score INTENTIGER integer
);

/*
3 rows from essays table:
essay_id	full_text	score
000d118	Many people have car where they live. The thing they don't know is that when you use a car alot of the time, you get the... 3
000fe60	I am a scientist at NASA that is discussing the face on mars. I will be explaining how the face is a... 3
001ab80	People always wish they had the same t

### **Model Evaluation**

In [43]:
from langchain.evaluation.criteria import LabeledCriteriaEvalChain

**1. Correctness**

In [44]:
# Assessing the 1st model: Gemma2

criteria = "correctness"
evaluator = LabeledCriteriaEvalChain.from_llm(llm=llm,criteria=criteria)
evaluator.evaluate_strings(
    prediction="Result: [(13,)]",
    input="How many essays have a score of 3 or more ?",
    reference="The number of  essays that have a score of 3 or more is equal to thirteen",)

{'reasoning': "1. **Correctness:**  We need to determine if the submission's result (13) is accurate and matches the reference information (thirteen essays with a score of 3 or more).  \n\n2. Both the submission and the reference state that there are thirteen essays with a score of 3 or more. \n\n\nY",
 'value': 'Y',
 'score': 1}